In [1]:
import magritte.core as magritte

In [2]:
magritte.pcmp_comm_rank()

0

In [3]:
import numpy as np

In [4]:
import healpy

In [5]:
nrays = 12*1**2

In [6]:
x,y,z = healpy.pix2vec(healpy.npix2nside(nrays),range(nrays))

In [7]:
Rs = np.array((x,y,z)).T

In [8]:
Rs

array([[ 5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01, -5.27046277e-01,  6.66666667e-01],
       [ 5.27046277e-01, -5.27046277e-01,  6.66666667e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.12323400e-17,  1.00000000e+00,  0.00000000e+00],
       [-1.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-1.83697020e-16, -1.00000000e+00,  0.00000000e+00],
       [ 5.27046277e-01,  5.27046277e-01, -6.66666667e-01],
       [-5.27046277e-01,  5.27046277e-01, -6.66666667e-01],
       [-5.27046277e-01, -5.27046277e-01, -6.66666667e-01],
       [ 5.27046277e-01, -5.27046277e-01, -6.66666667e-01]])

In [16]:
def load_balance_directions(directions, comm_size):
    """
    Basic attempt to imporved load balancing between different MPI
    processes by reordering the rays such that each process gets a
    similar set of direcitons.
    """
    # Initialize
    Rs = directions
    
    # Precompute distances between all directions
    distances = np.arccos(np.matmul(Rs, Rs.T))

    # Get antipode for each direction
    antipodes = np.argmax(distances, axis=0)

    # Restrict to one hemisphere (antipodes will follow)
    Rs = Rs[:len(Rs)//2]

    # Precompute distances between all directions
    distances = np.arccos(np.matmul(Rs, Rs.T))

    # Initialize index lists
    inds   = list(range(len(Rs)))
    inds_o = [[] for _ in range(comm_size)]

    while len(inds) >= comm_size:
        # Take the next index
        id0 = inds[0]
        inds     .remove(id0)
        inds_o[0].append(id0)
        # Append indices of close directions to other processes
        for i in range(1, comm_size):
            idi = inds[np.argmin(distances[id0][inds])]
            inds     .remove(idi)
            inds_o[i].append(idi)

    # Append final indices
    for i, idx in enumerate(inds):
        inds_o[i].append(idx)
        
    # Unravel
    inds_o = [j for sl in inds_o for j in sl]

    # Add antipodes
    inds_o.extend(antipodes[inds_o].tolist())
    
    return directions[inds_o]

In [19]:
load_balance_directions(Rs, 2)[:6]

array([[ 5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01, -5.27046277e-01,  6.66666667e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.12323400e-17,  1.00000000e+00,  0.00000000e+00],
       [ 5.27046277e-01, -5.27046277e-01,  6.66666667e-01]])

In [18]:
load_balance_directions(Rs, 2)[6:]

array([[-5.27046277e-01, -5.27046277e-01, -6.66666667e-01],
       [ 5.27046277e-01, -5.27046277e-01, -6.66666667e-01],
       [ 5.27046277e-01,  5.27046277e-01, -6.66666667e-01],
       [-1.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-1.83697020e-16, -1.00000000e+00,  0.00000000e+00],
       [-5.27046277e-01,  5.27046277e-01, -6.66666667e-01]])

In [13]:
distances[index]

array([1.49011612e-08, 1.11024234e+00, 1.68213734e+00, 1.11024234e+00,
       1.01567516e+00, 1.01567516e+00])

In [16]:
inds

[1, 2, 3, 4, 5]

4

In [15]:
inds_o

[[0], [], [], []]

In [22]:
index

0

In [20]:
max(distances[0])

3.141592653589793

In [21]:
antipodes = np.argmax(distances, axis=0)

In [22]:
Rs

array([[ 5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01, -5.27046277e-01,  6.66666667e-01],
       [ 5.27046277e-01, -5.27046277e-01,  6.66666667e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.12323400e-17,  1.00000000e+00,  0.00000000e+00],
       [-1.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-1.83697020e-16, -1.00000000e+00,  0.00000000e+00],
       [ 5.27046277e-01,  5.27046277e-01, -6.66666667e-01],
       [-5.27046277e-01,  5.27046277e-01, -6.66666667e-01],
       [-5.27046277e-01, -5.27046277e-01, -6.66666667e-01],
       [ 5.27046277e-01, -5.27046277e-01, -6.66666667e-01]])

In [24]:
Rs[antipodes] + Rs

array([[-1.11022302e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.11022302e-16,  0.00000000e+00],
       [-1.11022302e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.11022302e-16,  0.00000000e+00],
       [ 0.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-1.22464680e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-1.22464680e-16,  0.00000000e+00,  0.00000000e+00],
       [-1.11022302e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.11022302e-16,  0.00000000e+00],
       [-1.11022302e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.11022302e-16,  0.00000000e+00]])

In [18]:
Rs[inds_o[0:3]]

array([[ 0.03607264,  0.03607264,  0.99869792],
       [-0.99879546,  0.04906767,  0.        ],
       [ 0.99879546,  0.04906767,  0.        ]])

In [30]:
inds = list(range(len(Rs)))

In [31]:
inds_o = [inds.pop(0)]

In [32]:
inds

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [33]:
inds_o

[0]

In [35]:
inds_o.append(inds.pop(1))

In [36]:
inds

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [37]:
inds_o

[0, 2]

In [41]:
for i, ind in enumerate(inds):

    dists[inds_o]

array([[1.49011612e-08, 1.11024234e+00, 1.68213734e+00, 1.11024234e+00,
        1.01567516e+00, 1.01567516e+00, 2.12591749e+00, 2.12591749e+00,
        1.45945531e+00, 2.03135032e+00, 3.14159265e+00, 2.03135032e+00],
       [1.68213734e+00, 1.11024234e+00, 0.00000000e+00, 1.11024234e+00,
        2.12591749e+00, 2.12591749e+00, 1.01567516e+00, 1.01567516e+00,
        3.14159265e+00, 2.03135032e+00, 1.45945531e+00, 2.03135032e+00]])

In [55]:
dist = np.sum(dists[inds_o][:,inds], axis=0)

In [58]:
np.argmax(dist)

6

In [ ]:
ind_farthest = np.argmax(dist)

In [ ]:
inds_o.append(inds.pop(ind_farthest))

In [61]:
dist

array([2.22048467, 2.22048467, 3.14159265, 3.14159265, 3.14159265,
       3.14159265, 4.60104797, 4.06270064, 4.60104797, 4.06270064])

In [62]:
dist[6]

4.601047966043726

In [11]:
# Remove first element form Rs and put it in ordered
Rs_ordered = [Rs.pop(0)]

In [12]:
for 

[[0.5270462766947299, 0.5270462766947298, 0.6666666666666666]]

In [13]:
len(Rs)

10

In [14]:
Rs

[[-0.5270462766947298, 0.5270462766947299, 0.6666666666666666],
 [-0.52704627669473, -0.5270462766947298, 0.6666666666666666],
 [0.5270462766947298, -0.52704627669473, 0.6666666666666666],
 [1.0, 0.0, 0.0],
 [6.123233995736766e-17, 1.0, 0.0],
 [-1.0, 1.2246467991473532e-16, 0.0],
 [-1.8369701987210297e-16, -1.0, 0.0],
 [0.5270462766947299, 0.5270462766947298, -0.6666666666666666],
 [-0.5270462766947298, 0.5270462766947299, -0.6666666666666666],
 [-0.52704627669473, -0.5270462766947298, -0.6666666666666666]]

In [8]:
Rs

array([[ 5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01,  5.27046277e-01,  6.66666667e-01],
       [-5.27046277e-01, -5.27046277e-01,  6.66666667e-01],
       [ 5.27046277e-01, -5.27046277e-01,  6.66666667e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.12323400e-17,  1.00000000e+00,  0.00000000e+00],
       [-1.00000000e+00,  1.22464680e-16,  0.00000000e+00],
       [-1.83697020e-16, -1.00000000e+00,  0.00000000e+00],
       [ 5.27046277e-01,  5.27046277e-01, -6.66666667e-01],
       [-5.27046277e-01,  5.27046277e-01, -6.66666667e-01],
       [-5.27046277e-01, -5.27046277e-01, -6.66666667e-01],
       [ 5.27046277e-01, -5.27046277e-01, -6.66666667e-01]])

In [15]:
dists = np.arccos(np.matmul(Rs, Rs.T))

In [24]:
dists - dists.T

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])